### 导入所需的包

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
!pip install transformers
import re
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from transformers import BertForTokenClassification, BertTokenizer
from transformers import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm, trange
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 导入训练好的模型

In [43]:
model=torch.load('/content/drive/MyDrive/finalModel')

### 数据清洗

In [44]:
tag_to_ix = {"D": 0,
             "P": 1, 
             "j": 2, 
             "M": 3,
             "B": 4,
             "E": 5,
             "0": 6,
             "[CLS]": 7,
             "[SEP]": 8,
             "[PAD]": 9}

ix_to_tag = {0: "D",
             1: "P",
             2: "j",
             3: "M",
             4: "B",
             5: "E",
             6: "0",
             7: "[CLS]",
             8: "[SEP]",
             9: "[PAD]"}

In [45]:
tokenizer = BertTokenizer.from_pretrained("ethanyt/guwenbert-base")

In [46]:
print("Is CUDA available: ", torch.cuda.is_available())
n_gpu = torch.cuda.device_count()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU numbers: ", n_gpu)
print("device_name: ", torch.cuda.get_device_name(0))

Is CUDA available:  True
GPU numbers:  1
device_name:  Tesla K80


##### 输入文本

In [67]:
# 先帝创业未半而中道崩殂，今天下三分，益州疲弊，此诚危急存亡之秋也。
# 然侍卫之臣不懈于内，忠志之士忘身于外者，盖追先帝之殊遇，欲报之于陛下也。
# text1 = "告天即位高皇帝将登宝位先于前一年之十二月百官劝进时上御新宫拜词于天"
# text2 = "其略曰惟我中国自宋运告终帝命真人于沙漠入中国为天下主百有余年今运亦终"
# 是时连阴，入明年元旦即晴。
# 至日，日光皎洁，合祭天地，上即位于南郊。
# 后值倾覆，受任于败军之际，奉命于危难之间，尔来二十有一年矣。
# 先帝知臣谨慎，故临崩寄臣以大事也。
# 李锜据浙西反，相者言于锜曰：“朱氏有奇相，当生天子。”
# 锜既死，后入掖庭，为郭太后侍儿。
# 至若春和景明，波澜不惊;上下天光，一碧万顷;沙鸥翔集，锦鳞游泳;岸芷汀兰，郁郁青青。
# 而或长烟一空，皓月千里，浮光耀金，静影沉壁。
import re
# wordsInput("至若春和景明，波澜不惊;上下天光，一碧万顷;沙鸥翔集，锦鳞游泳;岸芷汀兰，郁郁青青。",
#            "而或长烟一空，皓月千里，浮光耀金，静影沉壁。")

In [70]:
def token_maker(text11, text22):
  tokenized_text1 = tokenizer.encode(text11, add_special_tokens=True)
  tokenized_text2 = tokenizer.encode(text22, add_special_tokens=True)
  tokenized_texts=[]
  tokenized_texts.append(tokenized_text1)
  tokenized_texts.append(tokenized_text2)
  MAX_LEN = 64
  for tokenized_text in tokenized_texts:
    for i in range(MAX_LEN-len(tokenized_text)):
      tokenized_text.append(1)
  input_ids = tokenized_texts
  input_ids = torch.tensor(input_ids)
  return input_ids,tokenized_texts

#### 创建mask并预测

In [72]:
def mask_maker(input_ids):
  attention_masks = []
  attention_masks = [[float(i != 1) for i in input_id] for input_id in input_ids]
  attention_masks = torch.tensor(attention_masks)
  return attention_masks

def gpu_transfer(attention_masks, input_ids):
  attention_masks=attention_masks.to(device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
  input_ids=input_ids.to(device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
  return input_ids,attention_masks

def model_use(input_ids, attention_masks):
  outputs = model(input_ids=input_ids, 
                  attention_mask=attention_masks,
                  token_type_ids=None,
                  position_ids=None)

  scores = outputs[0].detach().cpu().numpy()

  pred_flat1 = np.argmax(scores[0], axis=1).flatten()
  pred_flat2 = np.argmax(scores[1], axis=1).flatten()
  list = []
  list.append(pred_flat1)
  list.append(pred_flat2)
  return list

In [89]:
def puc_adder(text1,text11,pred_flat1):
  print("原标点：",end="")
  print(text1)
  list_text1 = list(text11)
  flag = 0
  for i in range(len(text1)+1):
    if(pred_flat1[i] == 0):
      list_text1.insert(i+flag, "，")
      flag+=1
    elif(pred_flat1[i] == 1):
      list_text1.insert(i+flag, "、")
      flag+=1
    elif(pred_flat1[i] == 2):
      list_text1.insert(i+flag, "。")
      flag+=1
    elif(pred_flat1[i] == 3):
      list_text1.insert(i+flag, "：")
      flag+=1
    elif(pred_flat1[i] == 4):
      list_text1.insert(i+flag, "：“")
      flag+=1
    elif(pred_flat1[i] == 5):
      list_text1.insert(i+flag, "。”")
      flag+=1
  print("加标点：",end="")
  print("".join(list_text1))

In [90]:
def puc(a, b):
  text1 = ""
  text2 = ""
  text11 = ""
  text22 = ""
  text1 = a
  text2 = b
  pattern = re.compile(r'[^\u4e00-\u9fa5]')
  text11 = re.sub(pattern, '', a)
  text22 = re.sub(pattern, '', b)
  input_idss,tokenized_texts = token_maker(text11, text22)
  attention_maskss = mask_maker(tokenized_texts)
  input_ids,attention_masks = gpu_transfer(attention_maskss, input_idss)
  pred_flat1 = model_use(input_ids, attention_masks)[0]
  pred_flat2 = model_use(input_ids, attention_masks)[1]
  puc_adder(text1, text11, pred_flat1)
  puc_adder(text2, text22, pred_flat2)
puc("庆历四年春，滕子京谪守巴陵郡。越明年，政通人和，百废具兴，乃重修岳阳楼，增其旧制，刻唐贤今人诗赋于其上，属予作文以记之。","俱予观夫巴陵胜状，在洞庭一湖。衔远山，吞长江，浩浩汤汤，横无际涯，朝晖夕阴，气象万千，此则岳阳楼之大观也，前人之述备矣。")

原标点：庆历四年春，滕子京谪守巴陵郡。越明年，政通人和，百废具兴，乃重修岳阳楼，增其旧制，刻唐贤今人诗赋于其上，属予作文以记之。
加标点：庆历四年春，滕子京谪守巴陵郡越，明年，政通人和，百废具兴，乃重修岳阳楼，增其旧制，刻唐贤今人诗赋于其上，属予作文以记之。
原标点：俱予观夫巴陵胜状，在洞庭一湖。衔远山，吞长江，浩浩汤汤，横无际涯，朝晖夕阴，气象万千，此则岳阳楼之大观也，前人之述备矣。
加标点：俱予观夫巴陵胜状，在洞庭一湖，衔远，山吞长江，浩浩汤汤，横无际涯，朝晖夕阴，气象万千，此则岳阳楼之大观也，前人之述备矣。


In [91]:
prag = "庆历四年春，滕子京谪守巴陵郡。越明年，政通人和，百废具兴，乃重修岳阳楼，增其旧制，刻唐贤今人诗赋于其上，属予作文以记之。\
俱予观夫巴陵胜状，在洞庭一湖。衔远山，吞长江，浩浩汤汤，横无际涯，朝晖夕阴，气象万千，此则岳阳楼之大观也，前人之述备矣。\
然则北通巫峡，南极潇湘，迁客骚人，多会于此，览物之情，得无异乎？\
若夫淫雨霏霏，连月不开，阴风怒号，浊浪排空，日星隐曜，山岳潜形，商旅不行，樯倾楫摧，薄暮冥冥，虎啸猿啼。\
登斯楼也，则有去国怀乡，忧谗畏讥，满目萧然，感极而悲者矣。\
至若春和景明，波澜不惊，上下天光，一碧万顷，沙鸥翔集，锦鳞游泳，岸芷汀兰，郁郁青青。\
而或长烟一空，皓月千里，浮光跃金，静影沉璧，渔歌互答，此乐何极！登斯楼也，则有心旷神怡，宠辱偕忘，把酒临风，其喜洋洋者矣。\
嗟夫！予尝求古仁人之心，或异二者之为，何哉？不以物喜，不以己悲，居庙堂之高则忧其民，处江湖之远则忧其君。是进亦忧，退亦忧。\
然则何时而乐耶？其必曰先天下之忧而忧，后天下之乐而乐乎！噫！微斯人，吾谁与归？时六年九月十五日。"

prag = prag.replace("？", "。").replace("！", "。").replace(".", "。").replace("。“", "。”").replace("；", "，").replace("”。", "。”")
prag = re.sub("。”", "。”\n", prag)
parg = re.sub("。(?!”)", "。\n", prag)
list_prag = parg.splitlines(False)
step = 2
list__ = [list_prag[i:i+step] for i in range(0, len(list_prag), step)]
for pair in list__:
  if len(pair)==1:
    pair.append("")
  puc(pair[0], pair[1])

原标点：庆历四年春，滕子京谪守巴陵郡。
加标点：庆历四年春，滕子京谪守巴陵郡。
原标点：越明年，政通人和，百废具兴，乃重修岳阳楼，增其旧制，刻唐贤今人诗赋于其上，属予作文以记之。
加标点：越明年，政通人和，百废具兴，乃重修岳阳楼，增其旧制，刻唐贤今人诗赋于其上，属予作文以记之。
原标点：俱予观夫巴陵胜状，在洞庭一湖。
加标点：俱予观夫巴陵胜状，在洞庭一湖。
原标点：衔远山，吞长江，浩浩汤汤，横无际涯，朝晖夕阴，气象万千，此则岳阳楼之大观也，前人之述备矣。
加标点：衔远山吞长江，浩浩汤汤，横无际涯，朝晖夕阴，气象万千，此则岳阳楼之大观也，前人之述备矣。
原标点：然则北通巫峡，南极潇湘，迁客骚人，多会于此，览物之情，得无异乎。
加标点：然则北通巫峡，南极潇湘，迁客骚人，多会于此，览物之情，得无异乎。
原标点：若夫淫雨霏霏，连月不开，阴风怒号，浊浪排空，日星隐曜，山岳潜形，商旅不行，樯倾楫摧，薄暮冥冥，虎啸猿啼。
加标点：若夫淫雨霏霏，连月不开，阴风怒号，浊浪排空，日星隐曜，山岳潜形，商旅不行，樯倾楫摧，薄暮冥冥，虎啸猿啼。
原标点：登斯楼也，则有去国怀乡，忧谗畏讥，满目萧然，感极而悲者矣。
加标点：登斯楼也，则有去国怀乡，忧谗畏讥，满目萧然，感极而悲者矣。
原标点：至若春和景明，波澜不惊，上下天光，一碧万顷，沙鸥翔集，锦鳞游泳，岸芷汀兰，郁郁青青。
加标点：至若春和景明，波澜不惊，上下天光，一碧万顷，沙鸥翔集，锦鳞游泳，岸芷汀兰，郁郁青青。
原标点：而或长烟一空，皓月千里，浮光跃金，静影沉璧，渔歌互答，此乐何极。
加标点：而或长烟一空，皓月千里，浮光跃金，静影沉璧，渔歌互答，此乐何极。
原标点：登斯楼也，则有心旷神怡，宠辱偕忘，把酒临风，其喜洋洋者矣。
加标点：登斯楼也，则有心旷神怡，宠辱偕忘，把酒临风，其喜洋洋者矣。
原标点：嗟夫。
加标点：嗟夫。
原标点：予尝求古仁人之心，或异二者之为，何哉。
加标点：予尝求古仁人之心，或异二者之为，何哉。
原标点：不以物喜，不以己悲，居庙堂之高则忧其民，处江湖之远则忧其君。
加标点：不以物喜，不以己悲，居庙堂之高，则忧其民，处江湖之远，则忧其君。
原标点：是进亦忧，退亦忧。
加标点：是进亦忧，退亦忧。
原标点：然则何时而乐耶。
加标点：然则何时而乐耶。
原标点：其必曰先天下之忧而忧，后天下之乐而乐乎。
加标点：其必曰先天下之忧而忧